In [20]:
import torch
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric.datasets import HGBDataset
import numpy as np


In [ ]:
dataset = HGBDataset(root='data/', name='acm')
data = dataset[0]

Extracting data/acm/raw/data.zip
Processing...
Done!


In [22]:
data

HeteroData(
  paper={
    x=[3025, 1902],
    y=[3025],
    train_mask=[3025],
    test_mask=[3025],
  },
  author={ x=[5959, 1902] },
  subject={ x=[56, 1902] },
  term={ num_nodes=1902 },
  (paper, cite, paper)={ edge_index=[2, 5343] },
  (paper, ref, paper)={ edge_index=[2, 5343] },
  (paper, to, author)={ edge_index=[2, 9949] },
  (author, to, paper)={ edge_index=[2, 9949] },
  (paper, to, subject)={ edge_index=[2, 3025] },
  (subject, to, paper)={ edge_index=[2, 3025] },
  (paper, to, term)={ edge_index=[2, 255619] },
  (term, to, paper)={ edge_index=[2, 255619] }
)

In [ ]:
def create_multiplex_node_set(num_node_dict, node_types, edge_types):
    """
    Create a multiplex node set from the given node type counts.
    
    Args:
        num_node_dict (dict): A dictionary mapping node types to their counts.
        node_types (list): A list of node types.
        edge_types (list): A list of edge types.
    
    Returns:
        A set of nodes for multiplex graph representation - each node splits into k nodes (where k is the number of edge types).
    """
    pass
        

def create_multiplex_edge_layer(node_set, edge_type, edge_index):
    pass

def hetero_to_multiplex(hetero_data: HeteroData):
    pass